# Quickstart

## 1. Indexing: Load

We need to first load the blog post contents. We can use DocumentLoaders for this, which are objects that load in data from a source and return a list of Documents. A `Document` is an object with some `page_content` (str) and `metadata` (dict).

In this case we’ll use the WebBaseLoader, which uses `urllib` to load HTML from web URLs and `BeautifulSoup` to parse it to text. We can customize the HTML -> text parsing by passing in parameters to the `BeautifulSoup` parser via `bs_kwargs` (see BeautifulSoup docs). In this case only HTML tags with class “post-content”, “post-title”, or “post-header” are relevant, so we’ll remove all others.

In [4]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
from dotenv import load_dotenv
import chromadb

_ = load_dotenv()

# Create a persistent chromaDB client
client = chromadb.PersistentClient(path="C:\\Users\\jnsep\\OneDrive\\Desktop\\Projects\\ChromaDB_storage")

In [5]:
# client.reset() # Empties and completely resets the database. ⚠️ This is destructive and not reversible.

In [12]:
# Load a document from the web

# # Only keep post title, headers, and content from the full HTML.
bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

In [17]:
len(docs[0].page_content)

42824

In [9]:
# # # Load a pdf file
# from langchain_community.document_loaders import PyPDFLoader
#
# loader = PyPDFLoader("la_trama_de_la_vida.pdf")
# docs = loader.load_and_split()

In [13]:
len(docs[0].page_content)

42824

In [14]:
print(docs[0].page_content[:500])



      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In


## 2. Indexing: Split

Our loaded document is over 42k characters long. This is too long to fit in the context window of many models. Even for those models that could fit the full post in their context window, models can struggle to find information in very long inputs.

To handle this we’ll split the `Document` into chunks for embedding and vector storage. This should help us retrieve only the most relevant bits of the blog post at run time.

In this case we’ll split our documents into chunks of 1000 characters with 200 characters of overlap between chunks. The overlap helps mitigate the possibility of separating a statement from important context related to it. We use the RecursiveCharacterTextSplitter, which will recursively split the document using common separators like new lines until each chunk is the appropriate size. This is the recommended text splitter for generic text use cases.

We set `add_start_index=True` so that the character index at which each split Document starts within the initial Document is preserved as metadata attribute “start_index”.

In [56]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=4000, chunk_overlap=300, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)
len(all_splits)

213

In [90]:
type(all_splits[0])

langchain_core.documents.base.Document

In [57]:
len(all_splits[0].page_content)

464

In [61]:
all_splits[0].metadata

{'source': 'la_trama_de_la_vida.pdf', 'page': 0, 'start_index': 0}

## 3. Indexing: Store

Now we need to index our 66 text chunks so that we can search over them at runtime. The most common way to do this is to embed the contents of each document split and insert these embeddings into a vector database (or vector store). When we want to search over our splits, we take a text search query, embed it, and perform some sort of “similarity” search to identify the stored splits with the most similar embeddings to our query embedding. The simplest similarity measure is cosine similarity — we measure the cosine of the angle between each pair of embeddings (which are high dimensional vectors).

We can embed and store all of our document splits in a single command using the Chroma vector store and OpenAIEmbeddings model.

In [62]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

In [91]:
type(vectorstore)

langchain_community.vectorstores.chroma.Chroma

## 4. Retrieval and Generation: Retrieve

Now let’s write the actual application logic. We want to create a simple application that takes a user question, searches for documents relevant to that question, passes the retrieved documents and initial question to a model, and returns an answer.

First we need to define our logic for searching over documents. LangChain defines a Retriever interface which wraps an index that can return relevant `Documents` given a string query.

The most common type of `Retriever` is the VectorStoreRetriever, which uses the similarity search capabilities of a vector store to facilitate retrieval. Any `VectorStore` can easily be turned into a `Retriever` with `VectorStore.as_retriever()`:

In [63]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

In [64]:
retrieved_docs = retriever.invoke("What are the approaches to Task Decomposition?")

In [65]:
len(retrieved_docs)

6

In [66]:
print(retrieved_docs[0].page_content)

102. 
danov fue el primero en intentar la integración de los conceptos de organización,  
patrón y complejidad en una teoría de sistemas coherente. Los cibernéticos se centraron en los patrones de comunicación y control –en particular en las pautas de circularidad causal subyacentes en el concepto de retroalimentación–, y al ha
  
cerlo, fueron los primeros en distinguir claramente el patrón de organización de un sistema, de su estructura física.  Las 
<<piezas del rompecabezas >> que faltaban –el concepto de autoorganización 
y las nuevas matemáticas de la complejidad– han sido identificadas y analizadas a lo largo de los últimos veinte años del siglo XX. Una vez más, la noción de patrón ha sido fundamental para ambos acontecimientos. El concepto de autoorganiza-ción se originó en el reconocimiento de la red como patrón general de vida, refina-do posteriormente por Maturana y Varela en su concepto de autopoiesis. Las nue-vas matemáticas de la complejidad son esencialmente unas matemát

In [92]:
type(retriever)

langchain_core.vectorstores.VectorStoreRetriever

## 5. Retrieval and Generation: Generate

Let’s put it all together into a chain that takes a question, retrieves relevant documents, constructs a prompt, passes that to a model, and parses the output.

We’ll use the gpt-3.5-turbo OpenAI chat model, but any LangChain `LLM` or `ChatModel` could be substituted in.

In [67]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125", temperature=0)

In [68]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [69]:
example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()
example_messages

[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: filler question \nContext: filler context \nAnswer:")]

In [70]:
print(example_messages[0].content)

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: filler question 
Context: filler context 
Answer:


In [71]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# rag_chain = (
#     {"context": retriever | format_docs, "question": RunnablePassthrough()}
#     | prompt
#     | llm
#     | StrOutputParser()
# )

In [72]:
# for chunk in rag_chain.stream("What is Task Decomposition?"):
#     print(chunk, end="", flush=True)

In [85]:
from langchain_core.prompts import PromptTemplate

template = """ Usa el contexto proporcionado para responder la pregunta que del final.
Si no sabes la respuesta a la pregunta, simplemente responde "No sé", no intentes inventar una respuesta.
Si la información del cntexto no es suficiente para responder la pregunta, responde "No tengo información suficiente para responder".
Utiliza máximo cuatro (4) oraciones y responde de la manera más concisa posible.

{context}

Pregunta: {question}

Helpful Answer:"""
custom_rag_prompt = PromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

In [77]:
rag_chain.invoke("Explica la crisis de percepción")

'La crisis de percepción se refiere a la falta de comprensión y conciencia de la interconexión y la interdependencia de los problemas globales que enfrentamos en la actualidad. Esta crisis surge de una visión desfasada del mundo, que no es adecuada para abordar los desafíos de un mundo superpoblado y globalmente interconectado. Para superar esta crisis, es necesario un cambio radical en la percepción, el pensamiento y los valores de la sociedad.'

In [78]:
rag_chain.invoke("Cuál es una posible causa de la crisis de percepción?")

'Una posible causa de la crisis de percepción es la visión desfasada del mundo que suscriben la mayoría de las personas y grandes instituciones sociales.'

In [79]:
rag_chain.invoke("Cuáles son los criterios clave de un sistema vivo?")

'Los criterios clave de un sistema vivo son el patrón de organización, la estructura y el proceso vital. Estos criterios se basan en la autopoiesis, la estructura disipativa y la cognición.'

In [95]:
rag_chain.invoke("Cuál es la masa del sol?")

'No tengo información suficiente para responder.'

In [93]:
type(rag_chain)

langchain_core.runnables.base.RunnableSequence